In [1]:
from itertools import combinations

import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder


In [2]:
EXCLUDE = (
    "sub-206",
    "sub-755",
    "sub-206",
    "sub-12058",
    "sub-42",
    "sub-12009",
)

BEH_COLS = [
    "id",
    "site",
    "age",
    "sex",
    "cycle",
    "CTQ_global",
    "CTQ_EM",
    "CTQ_EV",
    "CTQ_KM",
    "CTQ_KV",
    "CTQ_SM",
    "CTQ_BV",
    "AUCg",
    "AUCi",
    "increase",
    "HR",
    "Affect",
]

CAT_MAP = {
    "site": {
        0: "regensburg",
        1: "mannheim",
        2: "cacnac",
        3: "chrono",
    },
    "sex": {
        0: "male",
        1: "female",
    },
    "cycle": {
        0: "male",
        1: "luteal",
        2: "pill",
        3: "menopause",
    },
}
BEH_PATH = "/mnt/datasets/stressimaging/behavioral/Data_all_structure.csv"

SITES = ("cacnac", "regensburg", "mannheim")

beh = pd.read_csv(BEH_PATH).rename({"VPN": "id"}, axis=1).drop(columns=["VPNr"])
beh = beh[~beh["id"].isin(EXCLUDE)]
beh["sex"].replace(CAT_MAP["sex"], inplace=True)
beh["site"].replace(CAT_MAP["site"], inplace=True)
beh["cycle"].replace(CAT_MAP["cycle"], inplace=True)

beh = beh.reset_index(drop=True)
# Make site, sex, and cycle categorical
for col in ["site", "sex", "cycle"]:
    beh[col] = pd.Categorical(beh[col])
# Remove subjects with missing data (e.g., age, site, sex and cortisol increase)
beh = beh.dropna(subset=["age", "site", "sex", "increase"], inplace=False).reset_index(
    drop=True
)
beh = beh[BEH_COLS]
beh

,id,site,age,sex,cycle,CTQ_global,CTQ_EM,CTQ_EV,CTQ_KM,CTQ_KV,CTQ_SM,CTQ_BV,AUCg,AUCi,increase,HR,Affect
0,sub-01,chrono,27.0,male,male,27.0,5.0,7.0,5.0,5.0,5.0,0.0,214.715631,82.964895,1.748952,101.2927,2.33
1,sub-02,chrono,37.0,male,male,31.0,5.0,10.0,5.0,6.0,5.0,0.0,287.956461,117.474981,2.951702,81.4348,1.33
2,sub-03,chrono,28.0,male,male,27.0,5.0,7.0,5.0,5.0,5.0,0.0,827.786895,0.206895,0.386204,92.7484,1.67
3,sub-04,chrono,30.0,male,male,25.0,5.0,5.0,5.0,5.0,5.0,2.0,185.605504,45.910001,0.926890,64.6897,1.00
4,sub-05,chrono,30.0,male,male,45.0,8.0,18.0,5.0,9.0,5.0,0.0,108.971597,19.592956,0.706202,62.4496,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,sub-12112,cacnac,45.0,female,luteal,41.0,9.0,13.0,8.0,6.0,5.0,0.0,540.060000,333.195000,0.511000,NaN,1.00
456,sub-12114,cacnac,24.0,female,luteal,31.0,8.0,6.0,5.0,5.0,7.0,0.0,641.430000,171.405000,-2.391000,NaN,2.00
457,sub-12115,cacnac,44.0,female,luteal,37.0,7.0,9.0,6.0,7.0,8.0,0.0,567.337500,301.657500,-0.623000,NaN,2.00
458,sub-12116,cacnac,27.0,female,pill,27.0,5.0,6.0,5.0,6.0,5.0,0.0,784.987500,268.567500,-2.759000,NaN,3.00


In [3]:
COVs = ["site", "sex", "age"]
MEASURES = [
    "CTQ_global",
    "AUCg",
    "AUCi",
    "increase",
    "HR",
    "Affect",
]


def correct_for_covariates(df, target, covariates):
    """Correct for covariates in a dataframe using linear regression.
    It returns the residuals of the linear regression model."""
    y = df[target].dropna()
    X = pd.DataFrame()
    for cov in covariates:
        if df[cov].dtype == "category":
            cov_df = (
                OneHotEncoder(drop="first")
                .fit_transform(df[cov].values.reshape(-1, 1))
                .toarray()
            )
        else:
            cov_df = df[cov].values.reshape(-1, 1)
        X = pd.concat([X, pd.DataFrame(cov_df)], axis=1)
    X = X.iloc[y.index]
    mdl = LinearRegression().fit(X, y)
    return y - mdl.predict(X)


def compute_zscore(df, target):
    """Compute z-scores for a target variable by site."""
    return df.groupby("site")[target].transform(lambda x: (x - x.mean()) / x.std())


beh_residualized = beh.copy()

# Generate all possible combinations with lengths 1, 2, and 3
cov_combins = []
for r in range(1, len(COVs) + 1):
    cov_combins.extend(combinations(COVs, r))

for measure in MEASURES:
    for cov_combin in cov_combins:
        # beh_residualized[f"{measure}_log"] = np.log(beh[measure])
        beh_residualized[f"{measure}_zscore"] = compute_zscore(beh, measure)
        beh_residualized[f"{measure}_{'_'.join(cov_combin)}"] = correct_for_covariates(
            beh, measure, cov_combin
        )

beh_residualized.to_csv(
    "/mnt/projects/stressimaging/stress-fs/data/beh/beh_resitualized_161124.csv",
    index=False,
)

In [13]:
# Check whether site correction has been done correctly!
# We should not see any significant predictions for site after site-related correction.
def check_residuals(df, iv):
    """Check the residuals of the linear regression model."""
    X = df[iv]
    if df[iv].dtype == "category":
        X = OneHotEncoder(drop="first").fit_transform(X.values.reshape(-1, 1)).toarray()
    for measure in MEASURES:
        targets = [measure] + [f"{measure}_{'_'.join(cov)}" for cov in cov_combins]
        targets += [f"{measure}_zscore"]
        for tar in targets:
            y = beh_residualized[tar].dropna()
            X_tar = X[y.index]
            X_tar = sm.add_constant(X_tar)
            mdl = sm.OLS(y, X_tar).fit()
            print(f"IV: {iv}, {tar}: {mdl.rsquared:.3f}, p = {mdl.f_pvalue:.3f}")


check_residuals(beh_residualized, "site")

IV: site, CTQ_global: 0.018, p = 0.024
IV: site, CTQ_global_site: 0.000, p = 1.000
IV: site, CTQ_global_sex: 0.016, p = 0.036
IV: site, CTQ_global_age: 0.017, p = 0.027
IV: site, CTQ_global_site_sex: 0.000, p = 1.000
IV: site, CTQ_global_site_age: 0.000, p = 1.000
IV: site, CTQ_global_sex_age: 0.016, p = 0.037
IV: site, CTQ_global_site_sex_age: 0.000, p = 1.000
IV: site, CTQ_global_zscore: 0.000, p = 1.000
IV: site, AUCg: 0.226, p = 0.000
IV: site, AUCg_site: -0.000, p = 1.000
IV: site, AUCg_sex: 0.237, p = 0.000
IV: site, AUCg_age: 0.227, p = 0.000
IV: site, AUCg_site_sex: -0.000, p = 1.000
IV: site, AUCg_site_age: -0.000, p = 1.000
IV: site, AUCg_sex_age: 0.237, p = 0.000
IV: site, AUCg_site_sex_age: 0.000, p = 1.000
IV: site, AUCg_zscore: -0.000, p = 1.000
IV: site, AUCi: 0.227, p = 0.000
IV: site, AUCi_site: -0.000, p = 1.000
IV: site, AUCi_sex: 0.238, p = 0.000
IV: site, AUCi_age: 0.226, p = 0.000
IV: site, AUCi_site_sex: 0.000, p = 1.000
IV: site, AUCi_site_age: -0.000, p = 1.000

In [14]:
# Check whether age correction has been done correctly!
# We should not see any significant predictions for age after age-related correction.
check_residuals(beh_residualized, "age")

IV: age, CTQ_global: 0.008, p = 0.076
IV: age, CTQ_global_site: 0.007, p = 0.093
IV: age, CTQ_global_sex: 0.007, p = 0.083
IV: age, CTQ_global_age: 0.000, p = 1.000
IV: age, CTQ_global_site_sex: 0.007, p = 0.093
IV: age, CTQ_global_site_age: 0.000, p = 1.000
IV: age, CTQ_global_sex_age: 0.000, p = 1.000
IV: age, CTQ_global_site_sex_age: 0.000, p = 1.000
IV: age, CTQ_global_zscore: 0.009, p = 0.058
IV: age, AUCg: 0.000, p = 0.840
IV: age, AUCg_site: 0.001, p = 0.557
IV: age, AUCg_sex: 0.000, p = 0.893
IV: age, AUCg_age: -0.000, p = 1.000
IV: age, AUCg_site_sex: 0.001, p = 0.627
IV: age, AUCg_site_age: -0.000, p = 1.000
IV: age, AUCg_sex_age: 0.000, p = 1.000
IV: age, AUCg_site_sex_age: 0.000, p = 1.000
IV: age, AUCg_zscore: 0.001, p = 0.436
IV: age, AUCi: 0.000, p = 0.880
IV: age, AUCi_site: 0.000, p = 0.862
IV: age, AUCi_sex: 0.000, p = 0.824
IV: age, AUCi_age: 0.000, p = 1.000
IV: age, AUCi_site_sex: 0.000, p = 0.948
IV: age, AUCi_site_age: -0.000, p = 1.000
IV: age, AUCi_sex_age: -0.

In [15]:
# Check whether sex correction has been done correctly!
# We should not see any significant predictions for sex after sex-related correction.
check_residuals(beh_residualized, "sex")

IV: sex, CTQ_global: 0.001, p = 0.517
IV: sex, CTQ_global_site: 0.000, p = 0.995
IV: sex, CTQ_global_sex: 0.000, p = 1.000
IV: sex, CTQ_global_age: 0.001, p = 0.595
IV: sex, CTQ_global_site_sex: 0.000, p = 1.000
IV: sex, CTQ_global_site_age: 0.000, p = 0.967
IV: sex, CTQ_global_sex_age: 0.000, p = 1.000
IV: sex, CTQ_global_site_sex_age: 0.000, p = 1.000
IV: sex, CTQ_global_zscore: 0.000, p = 0.861
IV: sex, AUCg: 0.002, p = 0.329
IV: sex, AUCg_site: 0.031, p = 0.000
IV: sex, AUCg_sex: 0.000, p = 1.000
IV: sex, AUCg_age: 0.002, p = 0.336
IV: sex, AUCg_site_sex: 0.000, p = 1.000
IV: sex, AUCg_site_age: 0.031, p = 0.000
IV: sex, AUCg_sex_age: 0.000, p = 1.000
IV: sex, AUCg_site_sex_age: 0.000, p = 1.000
IV: sex, AUCg_zscore: 0.034, p = 0.000
IV: sex, AUCi: 0.002, p = 0.293
IV: sex, AUCi_site: 0.031, p = 0.000
IV: sex, AUCi_sex: 0.000, p = 1.000
IV: sex, AUCi_age: 0.002, p = 0.288
IV: sex, AUCi_site_sex: 0.000, p = 1.000
IV: sex, AUCi_site_age: 0.031, p = 0.000
IV: sex, AUCi_sex_age: -0.000